In [5]:
# -*- coding: UTF-8 -*-
"""
数据合并
组合为宽表
"""

# 保证脚本与Python3兼容
from __future__ import print_function

import os   #读取数据文件
import sys
import pymysql 
from sqlalchemy import create_engine
 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split  #划分训练集测试集使用
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.mosaicplot import mosaic
from sklearn.linear_model import LogisticRegression ,LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer 
from sklearn import metrics
from sklearn.feature_extraction import DictVectorizer #特征转换器
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.model_selection import GridSearchCV
%matplotlib inline 

import warnings

warnings.filterwarnings("ignore")

def readData(path):
    """
    使用pandas读取数据
    """
    data = pd.read_csv(path)
    cols = list(data.columns.values)
    return data[cols]
 
   
    
if __name__ == "__main__":
    # 设置显示格式
    pd.set_option('display.width', 1000)
    homePath = os.path.dirname(os.path.abspath('__file__'))
    # Windows下的存储路径与Linux并不相同
    if os.name == "nt":
        dataPath = "%s\\cust_data.csv" % homePath
    else:
        dataPath = "%s/cust_data.csv" % homePath
    custdata = readData(dataPath)
    
    if os.name == "nt":
        dataPath = "%s\\call_data.csv" % homePath
    else:
        dataPath = "%s/call_data.csv" % homePath
    calldata = readData(dataPath)
    
    if os.name == "nt":
        dataPath = "%s\\dpi_data.csv" % homePath
    else:
        dataPath = "%s/dpi_data.csv" % homePath
    dpidata = readData(dataPath)
    
    if os.name == "nt":
        dataPath = "%s\\prd_data.csv" % homePath
    else:
        dataPath = "%s/prd_data.csv" % homePath
    prddata = readData(dataPath)
    
    if os.name == "nt":
        dataPath = "%s\\trmnl_data.csv" % homePath
    else:
        dataPath = "%s/trmnl_data.csv" % homePath
    trmnldata = readData(dataPath)
    
    if os.name == "nt":
        dataPath = "%s\\train_result.csv" % homePath
    else:
        dataPath = "%s/train_result.csv" % homePath
    traindata = readData(dataPath)
    
    
    #连接合并
    data1 = pd.merge(custdata,traindata,on='user',how='left')
    print(data1.shape)
    print(data1.head())
    print(data1.columns)
    print(data1.describe())
   
    
    data2 = pd.merge(data1,calldata,on='user',how='left')
    data3 = pd.merge(data2,dpidata,on='user',how='left')
    data4 = pd.merge(data3,prddata,on='user',how='left')
    alldata = pd.merge(data4,trmnldata,on='user',how='left')   
     
    print(alldata.shape)    
    #训练和测试标记
    train = alldata[alldata['user'].str.contains("train")]
    test = alldata[alldata['user'].str.contains("test")]
    print(train.shape)  
    print(test.shape)
    
    #保存数据
    #训练数据
    train.to_csv('train.csv',index=0) 
    #测试数据
    test.to_csv('test.csv',index =0)
    #所有数据
    alldata.to_csv('all.csv',index=0)
    

   
    

(402164, 10)
           user  cust_access_net_dt  credit_level  membership_level  gender  birth_date  cust_point  inet_pd_inst_cnt  star_level             label
0  train_238138        2.015063e+13          -1.0            9999.0     2.0  19680409.0      9004.0               5.0      1300.0  7C26FADD409BD4B9
1    test_17547        2.019031e+13          -1.0            9999.0    -1.0  19780919.0         0.0               4.0      2100.0               NaN
2  train_301877        2.006060e+13      100100.0            9999.0     1.0  19451222.0       357.0               4.0      1200.0  7C26FADD409BD4B9
3  train_109061        2.014071e+13          -1.0            9999.0     1.0  19730606.0      6272.0               5.0      1500.0  7C26FADD409BD4B9
4  train_199787        2.004110e+13          -1.0            9999.0     1.0  19640413.0       988.0               1.0      1400.0  7C26FADD409BD4B9
Index(['user', 'cust_access_net_dt', 'credit_level', 'membership_level', 'gender', 'birth_date', 'c

In [7]:
#填补年龄
data1=data1.copy()
data1['AGE'].fillna(0, inplace=True)  #填补空值
data1=data1.copy()
data.to_csv('train.csv',header=0,index=0) #不保存列名和索引